In [9]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
  api_version = "2024-06-01"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

It looks like you're referencing the beginning of "The Star-Spangled Banner," the national anthem of the United States. Would you like to know more about its history or significance?


In [29]:
def get_completion(system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0.5, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content


In [30]:
import pandas as pd
forecast_df = pd.DataFrame({'Record_ID':[1,2,3,4,5],
                            'Unit_Cost':[10,20,30,40,50],
                            'Number of Units':[10,80,8,5,8]})
forecast_df['Total Cost'] =forecast_df['Unit_Cost']*forecast_df['Number of Units']

actuals_df = pd.DataFrame({'Record_ID':[1,2,3,4,5],
                            'Unit_Cost':[8,22,25,42,48],
                            'Number of Units':[8,70,9,7,9]})

actuals_df['Total Cost'] = actuals_df['Unit_Cost']*actuals_df['Number of Units']

In [27]:
forecast_df.head()

,Record_ID,Unit_Cost,Number of Units,Total Cost
0,1,10,10,100
1,2,20,80,1600
2,3,30,8,240
3,4,40,5,200
4,5,50,8,400


In [28]:
actuals_df.head()

,Record_ID,Unit_Cost,Number of Units,Total Cost
0,1,8,8,64
1,2,22,70,1540
2,3,25,9,225
3,4,42,7,294
4,5,48,9,432


In [31]:
## ---------- Call the helper method

### 1. Set primary content or prompt text
system_prompt= f"""
You are a financial analysis assistant.
You help by comparing actual and forecast data for financial data, in detail and summarizing results. 
"""

### 2. Use that in the prompt template below
user_prompt = f"""
There are a set of expenses that are compiled under the travel and intertainment General Ledge account.
These expenses are composed of unit costs and number of units.

There is a forecast (also known as budget) of each item with the following values:
{forecast_df.to_dict()}

Similarly, actual results were reported with the following values:
{actuals_df.to_dict()}

Provide a brief analysis of the variance between actuals and forecast, highlighting the main causes of the variance.
Call out specific percentages or dollar values.
"""

In [32]:
response = get_completion(system_prompt,user_prompt)
print(response)

### Variance Analysis: Travel and Entertainment Expenses

#### Overview
The analysis compares the forecasted (budgeted) expenses against the actual expenses incurred. The key metrics include Unit Cost, Number of Units, and Total Cost.

### Summary of Forecast vs. Actual

| Record_ID | Forecast Unit Cost | Actual Unit Cost | Forecast Units | Actual Units | Forecast Total Cost | Actual Total Cost | Variance ($) | Variance (%) |
|-----------|---------------------|-------------------|----------------|--------------|---------------------|-------------------|--------------|--------------|
| 1         | $10                 | $8                | 10             | 8            | $100                | $64               | -$36         | -36%         |
| 2         | $20                 | $22               | 80             | 70           | $1600               | $1540             | -$60         | -3.75%       |
| 3         | $30                 | $25               | 8              | 9            | $2